# ENTREGABLE 4

# INSTRUCCIONES

Utilizar el archivo CSV (`dataset_banco_clean.csv`) con 45189 filas y 17 columnas y aplicar las técnicas de normalización del entregable 3.

In [1]:
# imports
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('dataset_banco_clean.csv')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143.0,yes,no,unknown,5,may,261.0,1,-1.0,0,unknown,no
1,44,technician,single,secondary,no,29.0,yes,no,unknown,5,may,151.0,1,-1.0,0,unknown,no
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5,may,76.0,1,-1.0,0,unknown,no
3,47,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5,may,92.0,1,-1.0,0,unknown,no
4,33,unknown,single,unknown,no,1.0,no,no,unknown,5,may,198.0,1,-1.0,0,unknown,no


# Objetivo

Generar un model de clasificación capaz de predecir la clase de flor en función de las carácterísticas del dataset

* Aplicar las técnicas oportunas de procesamiento de datos

* Generar split de los datos

* Valorar diferentes modelos de clasificación

* Comparación entre modelos

* Ensemble

* Métricas

* Conclusiones finales

## Label encoder

para que los modelos puedan tratar los datos vamos a pasar todas las columnas a numérico

In [3]:
from sklearn.preprocessing import LabelEncoder

columnas_encoding= ['job','marital','education','default','housing','loan','contact','month','poutcome','y']

le = LabelEncoder()

for columna in columnas_encoding:
    df[columna]= le.fit_transform(df[columna])

print(df)

       age  job  marital  education  default  balance  housing  loan  contact  \
0       58    4        1          2        0   2143.0        1     0        2   
1       44    9        2          1        0     29.0        1     0        2   
2       33    2        1          1        0      2.0        1     1        2   
3       47    1        1          3        0   1506.0        1     0        2   
4       33   11        2          3        0      1.0        0     0        2   
...    ...  ...      ...        ...      ...      ...      ...   ...      ...   
45184   51    9        1          2        0    825.0        0     0        0   
45185   71    5        0          0        0   1729.0        0     0        0   
45186   72    5        1          1        0   5715.0        0     0        0   
45187   57    1        1          1        0    668.0        0     0        1   
45188   37    2        1          1        0   2971.0        0     0        0   

       day  month  duration

## Separar la X de la y

In [4]:
X = df.drop(columns='y')
y=df['y']

print(X)
print(y)

       age  job  marital  education  default  balance  housing  loan  contact  \
0       58    4        1          2        0   2143.0        1     0        2   
1       44    9        2          1        0     29.0        1     0        2   
2       33    2        1          1        0      2.0        1     1        2   
3       47    1        1          3        0   1506.0        1     0        2   
4       33   11        2          3        0      1.0        0     0        2   
...    ...  ...      ...        ...      ...      ...      ...   ...      ...   
45184   51    9        1          2        0    825.0        0     0        0   
45185   71    5        0          0        0   1729.0        0     0        0   
45186   72    5        1          1        0   5715.0        0     0        0   
45187   57    1        1          1        0    668.0        0     0        1   
45188   37    2        1          1        0   2971.0        0     0        0   

       day  month  duration

## Realizar los splits

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=30/100, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test,y_test,test_size=15/30, random_state=42)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(31632, 16) (31632,)
(6778, 16) (6778,)
(6779, 16) (6779,)


## Normalización

In [6]:
from sklearn.preprocessing import MinMaxScaler

def normalizar(xtr, xvl, xts):
    scaler = MinMaxScaler()
    xtr_s = scaler.fit_transform(xtr)
    xvl_s = scaler.transform(xvl)
    xts_s = scaler.transform(xts)
    return xtr_s, xvl_s, xts_s

x_train_s, x_val_s, x_test_s = normalizar(X_train,X_val,X_test)

# Verificar
print(x_train_s.min(axis=0),x_train_s.max(axis=0))
print(x_val_s.min(axis=0),x_val_s.max(axis=0))
print(x_test_s.min(axis=0),x_test_s.max(axis=0))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0.         0.         0.         0.         0.         0.00739799
 0.         0.         0.         0.         0.         0.00025773
 0.         0.         0.         0.        ] [0.85714286 1.         1.         1.         1.         0.1395572
 1.         1.         1.         1.         1.         1.26726804
 0.94736842 0.96215596 1.1372549  1.        ]
[0.         0.         0.         0.         0.         0.01101109
 0.         0.         0.         0.         0.         0.00025773
 0.         0.         0.         0.        ] [0.93506494 1.         1.         1.         1.         0.20566855
 1.         1.         1.         1.         1.         0.97525773
 1.0877193  0.95412844 0.78431373 1.        ]


## Aplicar modelos

A continuación, procedo a probar direfentes modelos con el set de train y val para tener una primera visión de que modelos pueden trabajar mejor con el problema presentado. También podría hacer un random search o un grid search, que lo haré con los me mejor resultado de de los modelos seleccionados.

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


models = {
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(),
    "Support Vector Machine": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "XGBoost": XGBClassifier(),
    "Decision Tree": DecisionTreeClassifier() 
}


for name, model in models.items():
    model.fit(x_train_s, y_train)
    y_pred = model.predict(x_val_s)
    accuracy = accuracy_score(y_val, y_pred)
    print(f"{name}: Precisión en el conjunto de prueba: {accuracy}")


Random Forest: Precisión en el conjunto de prueba: 0.9058719386249631
Logistic Regression: Precisión en el conjunto de prueba: 0.8925936854529359
Support Vector Machine: Precisión en el conjunto de prueba: 0.8909707878430215
K-Nearest Neighbors: Precisión en el conjunto de prueba: 0.8912658601357333
Naive Bayes: Precisión en el conjunto de prueba: 0.8406609619356742
XGBoost: Precisión en el conjunto de prueba: 0.9085275892593685
Decision Tree: Precisión en el conjunto de prueba: 0.8741516671584538


De este primero análisis podemos detectar que los modelos de naive-Bayes y el arbol de decisión no son los modelos que mejor soluciona el problema. 

Vamos a hacer un grid search para los otros modelos a ver si conseguimos mejoras.

In [30]:
from sklearn.model_selection import GridSearchCV

models = {
    "XGBoost": (XGBClassifier(), {'n_estimators': [100, 200, 300], 'learning_rate': [0.01, 0.1, 0.2]}),
    "K-Nearest Neighbors": (KNeighborsClassifier(), {'n_neighbors': [3, 5, 7]}),
    "Support Vector Machine": (SVC(), {'C': [0.1, 1, 10], 'gamma': [0.1, 0.01, 0.001], 'kernel': ['rbf']}),
    "Logistic Regression": (LogisticRegression(), {'C': [0.1, 1, 10], 'solver': ['liblinear', 'lbfgs']}),
    "Random Forest": (RandomForestClassifier(), {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20]})
}

for name, (model, params) in models.items():
    grid_search = GridSearchCV(estimator=model, param_grid=params, cv=5, scoring='accuracy')
    grid_search.fit(x_train_s, y_train)
    
    # Obtener el mejor modelo
    best_model = grid_search.best_estimator_
    
    # Predecir en el conjunto de prueba
    y_pred = best_model.predict(x_val_s)
    
    # Calcular la precisión del modelo en el conjunto de prueba
    accuracy = accuracy_score(y_val, y_pred)
    
    # Imprimir resultados
    print(f"Modelo: {name}")
    print(f"Mejores hiperparámetros encontrados: {grid_search.best_params_}")
    print(f"Precisión en el conjunto de prueba: {accuracy}\n")

Modelo: XGBoost
Mejores hiperparámetros encontrados: {'learning_rate': 0.1, 'n_estimators': 100}
Precisión en el conjunto de prueba: 0.9082325169666569

Modelo: K-Nearest Neighbors
Mejores hiperparámetros encontrados: {'n_neighbors': 7}
Precisión en el conjunto de prueba: 0.8909707878430215

Modelo: Support Vector Machine
Mejores hiperparámetros encontrados: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
Precisión en el conjunto de prueba: 0.8924461493065801

Modelo: Logistic Regression
Mejores hiperparámetros encontrados: {'C': 10, 'solver': 'lbfgs'}
Precisión en el conjunto de prueba: 0.8924461493065801

Modelo: Random Forest
Mejores hiperparámetros encontrados: {'max_depth': 20, 'n_estimators': 200}
Precisión en el conjunto de prueba: 0.9079374446739451



Vemos que mediante el grid search tampoco mejoran mucho los resultados del primer barrido realizado por lo que procedo a realizar ensembles para ver si mejora la capacidad de predicción ya que la principal ventaja de los ensembles es que permiten reducir el sobreajuste. 

Los modelos que selecciono para el ensemble son el random forest y el XGBoost ya que son los modelos que mejor resultado han dado. 

## Ensembles

In [17]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score


rf_clf = RandomForestClassifier(n_estimators=200, max_depth=20)
xgb_clf = XGBClassifier(learning_rate=0.1, n_estimators=100)

rf_clf.fit(x_train_s, y_train)
xgb_clf.fit(x_train_s, y_train)

rf_pred = rf_clf.predict(x_val_s)
xgb_pred = xgb_clf.predict(x_val_s)

rf_f1 = f1_score(y_val, rf_pred)
xgb_f1 = f1_score(y_val, xgb_pred)

rf_accuracy = accuracy_score(y_val, rf_pred)
xgb_accuracy = accuracy_score(y_val, xgb_pred)

voting_clf = VotingClassifier(estimators=[('random_forest', rf_clf), ('xgboost', xgb_clf)], voting='soft')

voting_clf.fit(x_train_s, y_train)

ensemble_pred = voting_clf.predict(x_val_s)

ensemble_f1 = f1_score(y_val, ensemble_pred)

ensemble_accuracy = accuracy_score(y_val, ensemble_pred)

print(f'El F1-score del modelo Random Forest es: {rf_f1}')
print(f'El accuracy del modelo Random Forest es: {rf_accuracy}')
print(f'El F1-score del modelo XGBoost es: {xgb_f1}')
print(f'El accuracy del modelo XGBoost es: {xgb_accuracy}')
print(f'El F1-score del ensemble es: {ensemble_f1}')
print(f'El accuracy del ensemble es: {ensemble_accuracy}')


El F1-score del modelo Random Forest es: 0.5047923322683706
El accuracy del modelo Random Forest es: 0.9085275892593685
El F1-score del modelo XGBoost es: 0.5244648318042814
El accuracy del modelo XGBoost es: 0.9082325169666569
El F1-score del ensemble es: 0.5293657008613939
El accuracy del ensemble es: 0.9113307760401298


Podemos ver que el accuracy (precisión del modelo) del en ensemble es algo mejor que unicamente utilizando el modelo XGBoost o el Random Forest y además vemos que el f1-score (permite equilibrar la precisión teniendo en cuenta falsos positivos y falsos negativos) del ensemble es mejor que el de los modelos por separado pero aun así no es extremadamente alto. Un F1-score de 0.52 indica que la precisión y el recall están relativamente equilibrados, pero aún hay errores tanto en los falsos positivos como en los falsos negativos.

A continuación voy a realizar un ensemble de los modelos de regrasión logistica y de XGB classifier ya que daba bien resultado previamente.

In [20]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, f1_score

# Instanciar los modelos con los hiperparámetros dados
logistic_regression = LogisticRegression(C=10, solver='lbfgs')
xgboost = XGBClassifier(learning_rate=0.1, n_estimators=100)

# Entrenar los modelos
logistic_regression.fit(x_train_s, y_train)
xgboost.fit(x_train_s, y_train)

# Predecir con los modelos
lr_pred = logistic_regression.predict(x_val_s)
xgb_pred = xgboost.predict(x_val_s)

# Calcular el F1-score para cada modelo
lr_f1 = f1_score(y_val, lr_pred)
xgb_f1 = f1_score(y_val, xgb_pred)

# Calcular el accuracy para cada modelo
lr_accuracy = accuracy_score(y_val, lr_pred)
xgb_accuracy = accuracy_score(y_val, xgb_pred)

# Crear el ensemble de clasificación
ensemble = VotingClassifier([('logistic_regression', logistic_regression), ('xgboost', xgboost)], voting='soft')

# Entrenar el ensemble
ensemble.fit(x_train_s, y_train)

# Predecir con el ensemble
ensemble_pred = ensemble.predict(x_val_s)

# Calcular el F1-score para el ensemble
ensemble_f1 = f1_score(y_val, ensemble_pred)

# Calcular el accuracy para el ensemble
ensemble_accuracy = accuracy_score(y_val, ensemble_pred)

print(f'El F1-score del modelo Logistic Regression es: {lr_f1}')
print(f'El accuracy del modelo Logistic Regression es: {lr_accuracy}')
print(f'El F1-score del modelo XGBoost es: {xgb_f1}')
print(f'El accuracy del modelo XGBoost es: {xgb_accuracy}')
print(f'El F1-score del ensemble es: {ensemble_f1}')
print(f'El accuracy del ensemble es: {ensemble_accuracy}')


El F1-score del modelo Logistic Regression es: 0.3037249283667622
El accuracy del modelo Logistic Regression es: 0.8924461493065801
El F1-score del modelo XGBoost es: 0.5244648318042814
El accuracy del modelo XGBoost es: 0.9082325169666569
El F1-score del ensemble es: 0.4396551724137931
El accuracy del ensemble es: 0.9041015048686928


Vemos que el ensemble o ha mejorado en gran medida los modelos por separado por lo que opto por probar otro ensemble de SVC y XGBClassifier.

In [22]:
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, f1_score

svm = SVC(C=10, gamma=0.1, kernel='rbf')
xgboost = XGBClassifier(learning_rate=0.1, n_estimators=100)

svm.fit(x_train_s, y_train)
xgboost.fit(x_train_s, y_train)

svm_pred = svm.predict(x_val_s)
xgb_pred = xgboost.predict(x_val_s)

svm_f1 = f1_score(y_val, svm_pred)
xgb_f1 = f1_score(y_val, xgb_pred)
svm_accuracy = accuracy_score(y_val, svm_pred)
xgb_accuracy = accuracy_score(y_val, xgb_pred)


ensemble = VotingClassifier([('svm', svm), ('xgboost', xgboost)], voting='hard')

ensemble.fit(x_train_s, y_train)

ensemble_pred = ensemble.predict(x_val_s)

ensemble_f1 = f1_score(y_val, ensemble_pred)
ensemble_accuracy = accuracy_score(y_val, ensemble_pred)

print(f'El F1-score del modelo SVM es: {svm_f1}')
print(f'El accuracy del modelo SVM es: {svm_accuracy}')
print(f'El F1-score del modelo XGBoost es: {xgb_f1}')
print(f'El accuracy del modelo XGBoost es: {xgb_accuracy}')
print(f'El F1-score del ensemble es: {ensemble_f1}')
print(f'El accuracy del ensemble es: {ensemble_accuracy}')


El F1-score del modelo SVM es: 0.21528525296017223
El accuracy del modelo SVM es: 0.8924461493065801
El F1-score del modelo XGBoost es: 0.5244648318042814
El accuracy del modelo XGBoost es: 0.9082325169666569
El F1-score del ensemble es: 0.20331491712707184
El accuracy del ensemble es: 0.893626438477427


De todos los modelos y pruebas realizadas vemos que el mejor resultado encontrado el la aplicación del ensemble de los modelos XGBoost y random forest tanto en accuracy como en f1-score. Este modelo utiliza la técnica de aumento de gradiente, en el que múltiples modelos como árboles de decisión se combinan para construir un modelo más fuerte.

Vamos a ver ese modelo que resultados nos da para el set de test, que es la prueba definitiva en la que eliminamos tanto sesgos de los modelos como sesgos causados por la elección del modelo. 

## modelo seleccionado

In [18]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score

rf_clf = RandomForestClassifier(n_estimators=200, max_depth=20)
xgb_clf = XGBClassifier(learning_rate=0.1, n_estimators=100)

voting_clf = VotingClassifier(estimators=[('random_forest', rf_clf), ('xgboost', xgb_clf)], voting='soft')

voting_clf.fit(x_train_s, y_train)

ensemble_pred = voting_clf.predict(x_val_s)

ensemble_f1 = f1_score(y_val, ensemble_pred)
ensemble_accuracy = accuracy_score(y_val, ensemble_pred)
print(f'El F1-score del ensemble es: {ensemble_f1}')
print(f'El accuracy del ensemble es: {ensemble_accuracy}')


El F1-score del ensemble es: 0.528125
El accuracy del ensemble es: 0.9108881676010623


In [19]:
ensemble_test = voting_clf.predict(x_test_s)

xgb_f1 = f1_score(y_test, ensemble_test)
xgb_accuracy = accuracy_score(y_test, ensemble_test)

print(f'El F1-score del ensemble en el set de test es: {xgb_f1}')
print(f'El accuracy del ensemble en el set de test es: {xgb_accuracy}')

El F1-score del ensemble en el set de test es: 0.5273408239700375
El accuracy del ensemble en el set de test es: 0.9069184245463933


# Conclusiones

A modo de conclusión decir que tenemos un modelo que tiene un accuracy alto lo que indica que el 90% de los datos que predice son correctos pero tiene un f1-score de 0,52 lo que indica que predice tanto falsos positivos como falsos negativos de manera equilibrada pero que seria interesante mejorar el parámetro aumentando el valor ya que en ese modo determinaría que el modelo es capaz de clasificar correctamente tanto instacias positivas como negativas. También destacar que se ha podido comprobar que el ensemble de modelos pueden llegar a hacer una predicción más robusta.